# TIN Geometry Parsing and Centroid Calculation

In [1]:
# -------------------------------------------------------------------------
# Author: Farid Javadnejad
# Date: 2025-02-05
# Description: 
# This script parses a TIN geometry from a LandXML file, computes triangle areas,
# and calculates the area-weighted centroid of the TIN.
# Functions:
#   - parse_tin_geometry: Parses the XML file to extract points and faces.
#   - compute_triangle_area: Computes the area of a triangle using the cross-product method.
#   - compute_weighted_centroid: Computes the area-weighted centroid of a TIN.
# -------------------------------------------------------------------------

## Libraries

In [2]:
import sys, re
from bs4 import BeautifulSoup as bs
import numpy as np

## Helper Functions

In [3]:
def parse_tin_geometry(xml_file, verbose=False):
    """
    Parses a TIN geometry from a LandXML file and returns points and faces.

    Steps:
    1. Reads the XML file using BeautifulSoup.
    2. Extracts faces from <F> tags, skipping those with i="1"
       read: <F n="5 2 6">9 5 6</F>
       return: [[9, 5, 6]]
    3. Extracts points from <P> tags.
       read: <P id="100">1.0 2.0 3.0</P>
       return: {100: np.array([1.0, 2.0, 3.0])}
    """
    points = {}
    faces = []

    # Read the XML file & catch any exception
    try:
        with open(xml_file, 'r', encoding='utf-8') as file:
            xml_soup = bs(file, 'lxml-xml')
    except Exception as e:
        raise ValueError(f"Error reading XML file: {e}")

    # Extract faces
    print("Extracting faces...")
    for face in xml_soup.find_all('F'):
        if face.get("i") == "1":
            continue  # Skip flagged faces
        face_list = []
        for x in face.text.split():
            face_list.append(int(x))
        faces.append(face_list)

        if verbose:
            # Print the face info
            print(face_list)
        

    # Extract points
    print("\nExtracting points...")
    for point_tag in xml_soup.find_all('P'):
        pid = int(point_tag["id"])
        point_list = []
        for x in point_tag.text.split():
            point_list.append(float(x))
        points[pid] = np.array(point_list)

        if verbose:
            # Print the point info
            print(point_list)
    
    # Return points and faces
    return points, faces


def compute_triangle_area(p1, p2, p3):
    """
    Computes the area of a triangle in 3D space using the cross-product method.
    Area = 0.5 * ||A x B|| = 0.5 ||(p2-p1) x (p3-p1)||
    """
    # Compute the vectors from point p1
    v1 = p2 - p1
    v2 = p3 - p1

    # Calculate the cross product
    cross_product = np.cross(v1, v2)

    # Compute the area using the magnitude of the cross product
    area = 0.5 * np.linalg.norm(cross_product)
    return area



def compute_weighted_centroid(points, faces, verbose=False):
    """ 
    Computes the area-weighted centroid of a Triangulated Irregular Network (TIN).
    1. Compute the centroid of each triangle
        Ci = (p1 + p2 + p3) / 3
    2. Compute the area of each triangle
        Ai = 0.5 * ||(p2-p1) x (p3-p1)||
    3. Compute the total weighted centroid
        Ctin = Σ(Ai * Ci) / Σ(Ai)
    """
    total_weighted_centroid = np.zeros(3)
    total_area = 0

    for face in faces:
        p1, p2, p3 = points[face[0]], points[face[1]], points[face[2]]
        area = compute_triangle_area(p1, p2, p3)     
        centroid = (p1 + p2 + p3) / 3.0
        total_weighted_centroid += area * centroid
        total_area += area

        if verbose:
            # Print the face info
            print(f"\nPoints: {p1}, {p2}, {p3}")
            print(f"Centroid: {centroid}")
            print(f"Area: {area}")
        
    weighted_centroid = total_weighted_centroid / total_area if total_area else None
    return weighted_centroid

## Main

In [5]:
# Load the XML file
xml_file = 'c:/Farid/Github Repositories/TIN centroids/tin_centroids/data/sample/my_tin.xml'

#parse data from XML file
points, faces = parse_tin_geometry(xml_file, verbose=True)

# Compute area-weighted centroid
weighted_centroid = compute_weighted_centroid(points, faces, True)
print("\nArea-weighted centroid:", weighted_centroid)


Extracting faces...
[1, 2, 3]
[1, 3, 4]
[2, 3, 5]

Extracting points...
[0.0, 0.0, 0.0]
[10.0, 3.0, 1.0]
[5.0, 4.0, 2.0]
[0.0, 8.0, 0.0]
[10.0, 6.0, 1.0]

Points: [0. 0. 0.], [10.  3.  1.], [5. 4. 2.]
Centroid: [5.         2.33333333 1.        ]
Area: 14.611639196202457

Points: [0. 0. 0.], [5. 4. 2.], [0. 8. 0.]
Centroid: [1.66666667 4.         0.66666667]
Area: 21.540659228538015

Points: [10.  3.  1.], [5. 4. 2.], [10.  6.  1.]
Centroid: [8.33333333 4.33333333 1.33333333]
Area: 7.648529270389178

Area-weighted centroid: [3.94278026 3.50221894 0.89427803]
